In [1]:
import sys
sys.path.append("D:\\WETrak")

import os
import numpy as np
from scipy.signal import savgol_filter
os.environ['KMP_DUPLICATE_LIB_OK']='True'

## 1. Helper function for GT

In [2]:
def parse_leap_all_finger(leap_data,finger_type_list):
    # (one file)
    pitch_yaw_values = []
    for i,per_leap_data in enumerate(leap_data):
        pitch_yaw_values_by_finger = []
        if "bone_angles" not in per_leap_data["left_hand"]:
            continue
            
        for finger_keys in finger_type_list:
            finger = per_leap_data["left_hand"]["bone_angles"][finger_keys]
            for bone_keys in ["Distal", "Intermediate", "Proximal"]:
                pitch_yaw_values_by_finger.append(finger[bone_keys]["Pitch"])
            pitch_yaw_values_by_finger.append(finger["Proximal"]["Yaw"])
    
        pitch_yaw_values.append(pitch_yaw_values_by_finger)
    
    pitch_yaw_values = np.array(pitch_yaw_values)
    new_pitch_yaw_values = pitch_yaw_values.reshape((pitch_yaw_values.shape[0],len(finger_type_list),4))
    return new_pitch_yaw_values

In [3]:
def smooth_gt(leap_data_list,finger_type_list):
    min_vals_list = []
    max_vals_list = []
    for finger_type in finger_type_list:
        min_vals,max_vals = default_min_max(finger_type)
        min_vals_list += min_vals
        max_vals_list += max_vals
    max_vals_list = [x - 0.1 for x in max_vals_list] #为了让smooth后的值包括在default范围内
    
    new_leap_data_list = []
    for idx,leap_data in enumerate(leap_data_list):    
        smoothAngles = np.zeros_like(leap_data)
        # window_length即窗口长度
        # 取值为奇数且不能超过len(x)。
        # 它越大，则平滑效果越明显；越小，则更贴近原始曲线。
        # polyorder为多项式拟合的阶数。
        # 它越小，则平滑效果越明显；越大，则更贴近原始曲线。
        windowsize = 199
        poly_deg = 6

        for i in range(leap_data.shape[1]):
            smoothAngles[:,i] = savgol_filter(leap_data[:,i], windowsize, poly_deg)
        
        smoothAngles = np.clip(smoothAngles, min_vals_list, max_vals_list)    
        new_leap_data_list.append(smoothAngles)
        
    return new_leap_data_list

In [4]:
def gt_normalize(leap_data_list,finger_type_list):
    ##将gt归一化到[0,1]，记录每2s窗口的gt的最大值、最小值
    min_vals_list = []
    max_vals_list = []
    new_leap_data_list = []
    for leap_data in leap_data_list:
        min_vals = np.min(leap_data, axis=0) #每一段2s数据的最小值, eg,4 or 12
        max_vals = np.max(leap_data, axis=0) #每一段2s数据的最大值, eg,4 or 12
        # 归一化后的leap data
        new_leap_data = (leap_data - min_vals) / (max_vals - min_vals)
        
        min_vals_list.append(min_vals) #所有条数据的最小值
        max_vals_list.append(max_vals) #所有条数据的最大值
        new_leap_data_list.append(new_leap_data)
    
    return np.array(min_vals_list),np.array(max_vals_list),np.array(new_leap_data_list)

In [5]:
def default_min_max(finger_type):
    ##将理论运动范围转为从0开始 （方便regression）
    if finger_type == "Thumb":
        # DIP [-10,80] -> [-35,60]
        # MCP-pitch [-15,70] -> [-40,45]
        # MPC-yaw [-30,30] -> [-20,40+10]
        start = [-35,-20,-40,-20]
        end = [60,60,45,50]
        
    elif finger_type == "Index":
        start = [-5,-3,-40,-30]
        end = [90,110,90,30]
        
    elif finger_type == "Middle":
        start = [-5,-3,-40,-22.5]
        end = [90,110,90,22.5]
        
    elif finger_type == "Ring":
        start = [-5,-3,-40,-22.5]
        end = [90,120,90,22.5]
    
    elif finger_type == "Pinky":
        start = [-5,-3,-40,-25]
        end = [90,135,90,25]
        
    delta = -np.array(start)    
    min_vals = []
    max_vals = []
    for i in [0,1,2,3]:
        lower_bound = start[i] + delta[i]
        upper_bound = end[i] + delta[i]
        
        min_vals.append(lower_bound)
        max_vals.append(upper_bound)
    
    return min_vals,max_vals

In [6]:
def get_angle_template_default(finger_type_list,classify_num):
    default_min_vals_list = []
    default_max_vals_list = []
    
    for finger_type in finger_type_list:
        min_vals,max_vals = default_min_max(finger_type)

        default_min_vals_list += min_vals
        default_max_vals_list += max_vals

    default_min_vals_list += default_min_vals_list #前一半和后一半的default角度一样
    default_max_vals_list += default_max_vals_list

    # length: 24, 每个子模板也不一样
    angle_template = [[] for _ in range(len(default_min_vals_list))] 
    
    for idx in range(len(angle_template)):
        interval = (default_max_vals_list[idx] - default_min_vals_list[idx]) / classify_num

        angle_template[idx] = [[default_min_vals_list[idx] + i*interval, default_min_vals_list[idx] + (i+1)*interval] for i in range(classify_num)]

    return angle_template

In [7]:
def min_max_bin_index(min_max_data,finger_type_list,classify_num):
    # 记录min, max所在bin的下标，即哪一类
    angle_template = get_angle_template_default(finger_type_list,classify_num)

    target = []
    for idx in range(min_max_data.shape[0]):
        for j in range(len(angle_template[idx])):
            if min_max_data[idx] >= angle_template[idx][j][0] and min_max_data[idx] < angle_template[idx][j][1]:
                target.append(j)
                break

    one_hot = [[0 for _ in range(classify_num)] for _ in range(len(target))]
    for i, val in enumerate(target):
        one_hot[i][val] = 1

    return np.array(target),np.array(one_hot).T

## 2. Helper function for Emg

In [8]:
def emg_z_norm(emg_data_list):
    norm_emg_data_list = []
    for emg_data_raw in emg_data_list:
        norm_emg_data = []

        for i in range(emg_data_raw.shape[1]):
            emg_data = emg_data_raw[:,i]
            # normalize z-score 
            mu = np.mean(emg_data,axis=0)
            sigma = np.std(emg_data,axis=0)
            emg_data = (emg_data - mu)/sigma

            norm_emg_data.append(emg_data)
        
        norm_emg_data_list.append(np.array(norm_emg_data).T)
    
    return norm_emg_data_list